In [13]:
import pandas as pd
import gensim
from gensim.models import KeyedVectors, Word2Vec
import os
from ast import literal_eval
os.chdir('/Users/ethanwu/Desktop/Python/Fubon/04. momo_10k_prod')

gensim.__version__

'3.8.3'

In [25]:
data = pd.read_csv('./data/deployment/input_sample.csv',
                  converters = {'brand':str,
                               'element':literal_eval})
data['tag_result'] = [['WhiteOak','美國', '牛仔', '牛仔褲', '李維', '猶太', '商人', '服飾']]
data.head(3)

,brand,element,tag_result
0,Levis,[李维斯（Levi's）是著名的牛仔裤品牌，作为牛仔裤的“鼻祖”，象征着美国野性、刚毅、叛逆...,"[WhiteOak, 美國, 牛仔, 牛仔褲, 李維, 猶太, 商人, 服飾]"


In [26]:
%%time
model = KeyedVectors.load_word2vec_format('./model/w2v_CNA_ASBC_300d.vec',
                                        binary=False, encoding = 'utf-8', unicode_errors='ignore')

CPU times: user 55.6 s, sys: 970 ms, total: 56.6 s
Wall time: 56.1 s


In [28]:
complement_data = set()
for x in data['tag_result'][0]:
    if x not in model.vocab:
        complement_data.add(x)

#[x]是gensim格式
complement_data = [[x] for x in complement_data]
complement_data

[['WhiteOak']]

#### Transfer Learning

In [30]:
%%time 
#lockf=0.0(不更新)->1.0(更新)

model_new = gensim.models.Word2Vec(min_count=1,size=300)
model_new.build_vocab([[x] for x in model.vocab.keys()]+complement_data)
model_new.intersect_word2vec_format('./model/w2v_CNA_ASBC_300d.vec',
                                   lockf=0.0,binary=False,unicode_errors='ignore')
model_new.train(data['tag_result'].to_list(),total_examples=(len(data['tag_result'].to_list()
                                                                +complement_data)),
                epochs=30)

CPU times: user 2min 1s, sys: 1.75 s, total: 2min 3s
Wall time: 2min 2s


(240, 240)

In [32]:
print(f'Origin:',model['戒指'][:10])
print(f'New:',model_new.wv['戒指'][:10])

Origin: [-0.100454  0.580426  0.08833  -0.233083  0.155034  0.328045 -0.201915
  0.204724  0.403323 -0.240968]
New: [-0.100454  0.580426  0.08833  -0.233083  0.155034  0.328045 -0.201915
  0.204724  0.403323 -0.240968]


In [33]:
model_new.wv.most_similar('WhiteOak')

[('牛仔', 0.720608115196228),
 ('牛仔褲', 0.7087457776069641),
 ('李維', 0.6976678371429443),
 ('襯衫', 0.5930806398391724),
 ('穿著', 0.5644718408584595),
 ('褲', 0.5548107028007507),
 ('立領', 0.5545482039451599),
 ('穿', 0.5527833104133606),
 ('上衣', 0.5522429943084717),
 ('綠惡魔', 0.5514758825302124)]

In [34]:
'WhiteOak' in model_new.wv.vocab

True

#### Note:
* 做完transfer learning, 直接存model再做get vector會比較單純
* 不建議做完transfer learning後存向量

In [ ]:
# Save Model:
# model_new.save('./model/w2v_CNA_ASBC_300d_new.model')
# Word2Vec.load('./model/w2v_CNA_ASBC_300d_new.model')

In [ ]:
# Save weight: 
# from gensim.models import KeyedVectors, Word2Vec
# model_new.save_word2vec_format('./model/w2v_CNA_ASBC_300d_new.bin',binary=True)
# model = KeyedVectors.load_word2vec_format('./model/w2v_CNA_ASBC_300d_new.bin',
#                                         binary=True, encoding = 'utf-8', unicode_errors='ignore')